<a href="https://colab.research.google.com/github/allenai/vampire/blob/colab/colab/VAMPIRE_AGNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/allenai/vampire


fatal: destination path 'vampire' already exists and is not an empty directory.


In [2]:
%cd vampire

/content/vampire


In [3]:
!pip install -r requirements.txt

  Cloning git://github.com/allenai/allennlp (to revision master) to /tmp/pip-req-build-dry2nuzv
  Running command git clone -q git://github.com/allenai/allennlp /tmp/pip-req-build-dry2nuzv
  Stored in directory: /tmp/pip-ephem-wheel-cache-k0g8loph/wheels/1f/68/bb/eb201425cf5c8214096ae0fceb531500ec6f9b6929f752eae2
Successfully built allennlp


In [4]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [5]:
!SEED=42 python -m pytest -v --color=yes vampire

============================= test session starts ==============================
platform linux -- Python 3.6.7, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/vampire, inifile:
plugins: flaky-3.5.3
collected 18 items                                                             

vampire/tests/data/dataset_readers/semisupervised_text_classification_json_test.py::TestSemiSupervisedTextClassificationJsonReader::test_ignores_label_properly PASSED [  5%]
vampire/tests/data/dataset_readers/semisupervised_text_classification_json_test.py::TestSemiSupervisedTextClassificationJsonReader::test_read_from_file PASSED [ 11%]
vampire/tests/data/dataset_readers/semisupervised_text_classification_json_test.py::TestSemiSupervisedTextClassificationJsonReader::test_read_from_file_and_truncates_properly PASSED [ 16%]
vampire/tests/data/dataset_readers/semisupervised_text_classification_json_test.py::TestSemiSupervisedTextClassificationJsonReader::test_sa

In [6]:
!sh scripts/download_ag.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31.3M  100 31.3M    0     0  24.9M      0  0:00:01  0:00:01 --:--:-- 24.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1397k  100 1397k    0     0  2422k      0 --:--:-- --:--:-- --:--:-- 2418k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1955k  100 1955k    0     0  3343k      0 --:--:-- --:--:-- --:--:-- 3343k


In [7]:
!curl -Lo ag.tar https://s3-us-west-2.amazonaws.com/allennlp/datasets/ag-news/vampire_preprocessed_example.tar
!tar -xvf ag.tar -C examples/
!rm ag.tar

rm: cannot remove 'examples/ag/ag.tar': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5063k  100 5063k    0     0  7874k      0 --:--:-- --:--:-- --:--:-- 7862k
ag/
ag/train.npz
ag/dev.npz
ag/vampire.bgfreq
ag/reference/
ag/vocabulary/
ag/vocabulary/labels.txt
ag/vocabulary/non_padded_namespaces.txt
ag/vocabulary/vampire.txt
ag/reference/ref.vocab.json
ag/reference/ref.npz


In [0]:
!DATA_DIR="$(pwd)/examples/ag" VOCAB_SIZE=30000 LAZY=1 python -m scripts.train \
            --config training_config/vampire.jsonnet \
            --serialization-dir model_logs/vampire \
            --environment VAMPIRE \
            --device 0 -o

overriding model_logs/vampire
2019-06-15 21:51:38,959 - INFO - pytorch_pretrained_bert.modeling - Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2019-06-15 21:51:40,049 - INFO - allennlp.common.params - random_seed = 34543
2019-06-15 21:51:40,049 - INFO - allennlp.common.params - numpy_seed = 34543
2019-06-15 21:51:40,049 - INFO - allennlp.common.params - pytorch_seed = 34543
2019-06-15 21:51:40,095 - INFO - allennlp.common.checks - Pytorch version: 1.1.0
2019-06-15 21:51:40,096 - INFO - allennlp.common.params - evaluate_on_test = False
2019-06-15 21:51:40,097 - INFO - allennlp.common.from_params - instantiating class <class 'allennlp.data.dataset_readers.dataset_reader.DatasetReader'> from params {'lazy': True, 'type': 'vampire_reader'} and extras set()
2019-06-15 21:51:40,097 - INFO - allennlp.common.params - dataset_reader.type = vampire_reader
2019-06-15 21:51:40,097 - INFO - allennlp.common.from_params - instantiating class <class 'vampi

In [0]:
!DATA_DIR="$(pwd)/examples/ag" VAMPIRE_DIR="$(pwd)/model_logs/vampire" VAMPIRE_DIM=81 THROTTLE=200 EVALUATE_ON_TEST=0 python -m scripts.train \
            --config training_config/classifier.jsonnet \
            --serialization-dir model_logs/clf \
            --environment CLASSIFIER \
            --device 0